📦 Importación y configuración inicial

Este bloque se encarga de preparar el entorno para llevar a cabo el análisis de datos. Aquí se añaden las rutas necesarias para importar los módulos internos del proyecto, se cargan bibliotecas como Pandas, se importan los modelos ORM de SQLAlchemy que representan las tablas de la base de datos, y se traen las funciones que se utilizarán para realizar el EDA y los chequeos de calidad.

Por último, se establece una sesión de base de datos, lo que permite consultar las tablas y convertirlas en DataFrames para analizarlas de manera más cómoda.

In [1]:
import sys
sys.path.append("..")

import pandas as pd

from M2_V1.db import SessionLocal
from M2_V1.models import (Usuario, Categoria, Producto, Orden, DetalleOrden, DireccionEnvio, Carrito, MetodoPago, OrdenMetodoPago, ResenaProducto, HistorialPago)

from analysis.eda_sqlalchemy import run_full_eda, table_to_df
from analysis.quality_checks import (null_count, null_percentage,duplicates_count, duplicated_rows,iqr_outliers, negative_values, invalid_categories)

session = SessionLocal()

Esta función crea un resumen rápido de toda la base de datos al consultar cada modelo ORM: recoge el nombre de la tabla, sus columnas y la cantidad de registros. Es útil como un diagnóstico inicial para asegurarse de que las tablas existen, se pueden leer correctamente y para tener una visión general antes de profundizar en análisis más detallados.

In [2]:
run_full_eda(session)

[{'table': 'usuarios',
  'columns': [('usuario_id', 'INTEGER'),
   ('nombre', 'VARCHAR(100)'),
   ('apellido', 'VARCHAR(100)'),
   ('dni', 'VARCHAR(20)'),
   ('email', 'VARCHAR(255)'),
   ('contrasena', 'VARCHAR(255)'),
   ('fecha_registro', 'DATETIME')],
  'rows': 1000},
 {'table': 'categorias',
  'columns': [('categoria_id', 'INTEGER'),
   ('nombre', 'VARCHAR(100)'),
   ('descripcion', 'VARCHAR(255)')],
  'rows': 12},
 {'table': 'productos',
  'columns': [('producto_id', 'INTEGER'),
   ('nombre', 'VARCHAR(255)'),
   ('descripcion', 'TEXT'),
   ('precio', 'NUMERIC(10, 2)'),
   ('stock', 'INTEGER'),
   ('categoria_id', 'INTEGER')],
  'rows': 36},
 {'table': 'ordenes',
  'columns': [('orden_id', 'INTEGER'),
   ('usuario_id', 'INTEGER'),
   ('fecha_orden', 'DATETIME'),
   ('total', 'NUMERIC(10, 2)'),
   ('estado', 'VARCHAR(50)')],
  'rows': 10000},
 {'table': 'detalleordenes',
  'columns': [('detalle_id', 'INTEGER'),
   ('orden_id', 'INTEGER'),
   ('producto_id', 'INTEGER'),
   ('cantida

1) Creación del diccionario de tablas y conversión a DataFrames

En este bloque, se establece un diccionario que vincula un nombre comprensible a cada modelo ORM, y luego se transforma cada una de esas tablas en DataFrames de pandas. Esto facilita el trabajo con toda la base de datos en un formato tabular, que es perfecto para el análisis. La meta es preparar los datos para realizar evaluaciones de calidad de manera consistente y evitar la repetición de código al acceder a cada tabla.

In [3]:
tables = {
    "usuarios": Usuario,
    "categorias": Categoria,
    "productos": Producto,
    "ordenes": Orden,
    "detalleordenes": DetalleOrden,
    "direccionesenvio": DireccionEnvio,
    "carrito": Carrito,
    "metodospago": MetodoPago,
    "ordenesmetodospago": OrdenMetodoPago,
    "resenasproductos": ResenaProducto,
    "historialpagos": HistorialPago,
}

dfs = {name: table_to_df(session, model) for name, model in tables.items()}

2) Generación del reporte de calidad por tabla

En este paso, se revisa cada DataFrame y se generan métricas básicas de calidad, como la cantidad de valores nulos, el porcentaje de nulos, la cantidad de duplicados y las filas duplicadas. El objetivo es obtener un diagnóstico inicial que sea comparable entre las tablas, permitiendo identificar rápidamente problemas estructurales que podrían afectar análisis futuros o la carga en un pipeline ETL.

In [4]:
quality_report = {}

for name, df in dfs.items():
    quality_report[name] = {
        "null_count": null_count(df),
        "null_percentage": null_percentage(df),
        "duplicates_count": duplicates_count(df),
        "duplicated_rows": duplicated_rows(df),
    }

3) Visualización estructurada del reporte

Este bloque presenta el reporte de calidad de manera ordenada para cada tabla. Se destacan los valores nulos, duplicados y otros indicadores en secciones bien definidas, lo que hace más fácil revisar manualmente los resultados y detectar cualquier anomalía. Es la etapa final del mini-pipeline de calidad, donde toda la información calculada anteriormente se muestra de forma clara y útil para auditorías o documentación.

In [5]:
for table, report in quality_report.items():
    print(f"\n===== {table.upper()} =====")

    print("\n--- Nulos ---")
    print(report["null_count"])

    print("\n--- Porcentaje de nulos ---")
    print(report["null_percentage"])

    print("\n--- Cantidad de duplicados ---")
    print(report["duplicates_count"])

    print("\n--- Filas duplicadas (si existen) ---")
    print(report["duplicated_rows"])
    print("\n---------------------------------------------")


===== USUARIOS =====

--- Nulos ---
usuario_id        0
nombre            0
apellido          0
dni               0
email             0
contrasena        0
fecha_registro    0
dtype: int64

--- Porcentaje de nulos ---
usuario_id        0.0
nombre            0.0
apellido          0.0
dni               0.0
email             0.0
contrasena        0.0
fecha_registro    0.0
dtype: float64

--- Cantidad de duplicados ---
0

--- Filas duplicadas (si existen) ---
Empty DataFrame
Columns: [usuario_id, nombre, apellido, dni, email, contrasena, fecha_registro]
Index: []

---------------------------------------------

===== CATEGORIAS =====

--- Nulos ---
categoria_id    0
nombre          0
descripcion     0
dtype: int64

--- Porcentaje de nulos ---
categoria_id    0.0
nombre          0.0
descripcion     0.0
dtype: float64

--- Cantidad de duplicados ---
0

--- Filas duplicadas (si existen) ---
Empty DataFrame
Columns: [categoria_id, nombre, descripcion]
Index: []

-------------------------------

En este apartado, se desarrolla un sistema integral de validación que detecta inconsistencias más profundas en los datos. Primero, se extraen las claves válidas de cada tabla maestra para verificar la integridad referencial. Luego, se establece una función auxiliar que registra cualquier problema que se encuentre. Con esto, se llevan a cabo controles específicos por tabla, que incluyen: valores negativos donde no deberían estar, outliers identificados mediante IQR, y claves foráneas inválidas que podrían afectar las relaciones del modelo. El resultado es un diccionario estructurado (quality_issues) que centraliza todas las anomalías detectadas, sirviendo como insumo final para decidir las acciones de limpieza o corrección dentro del pipeline.

In [15]:
usuarios_to_analize = dfs["usuarios"]["usuario_id"].tolist()
categorias_to_analize = dfs["categorias"]["categoria_id"].tolist()
productos_to_analize = dfs["productos"]["producto_id"].tolist()
metodos_to_analize = dfs["metodospago"]["metodo_pago_id"].tolist()
ordenes_to_analize = dfs["ordenes"]["orden_id"].tolist()

quality_issues = {}

def add_issue(table, column, result):
    if not result.empty:
        quality_issues.setdefault(table, {})[column] = result

# 🍎 PRODUCTOS

add_issue("productos", "precio_negative", negative_values(dfs["productos"], "precio"))
add_issue("productos", "stock_negative", negative_values(dfs["productos"], "stock"))

add_issue("productos", "precio_outliers", iqr_outliers(dfs["productos"], "precio"))
add_issue("productos", "stock_outliers", iqr_outliers(dfs["productos"], "stock"))

add_issue(
    "productos",
    "categoria_id_invalid",
    invalid_categories(dfs["productos"], "categoria_id", categorias_to_analize)
)

# 🍎 ORDENES

add_issue("ordenes", "usuario_id_invalid",
          invalid_categories(dfs["ordenes"], "usuario_id", usuarios_to_analize))

add_issue("ordenes", "total_negative",
          negative_values(dfs["ordenes"], "total"))

add_issue("ordenes", "total_outliers",
          iqr_outliers(dfs["ordenes"], "total"))

# 🍎 DETALLE ORDENES

add_issue("detalleordenes", "orden_id_invalid",
          invalid_categories(dfs["detalleordenes"], "orden_id", ordenes_to_analize))

add_issue("detalleordenes", "producto_id_invalid",
          invalid_categories(dfs["detalleordenes"], "producto_id", productos_to_analize))

add_issue("detalleordenes", "cantidad_negative",
          negative_values(dfs["detalleordenes"], "cantidad"))

add_issue("detalleordenes", "precio_unitario_negative",
          negative_values(dfs["detalleordenes"], "precio_unitario"))

add_issue("detalleordenes", "cantidad_outliers",
          iqr_outliers(dfs["detalleordenes"], "cantidad"))

add_issue("detalleordenes", "precio_unitario_outliers",
          iqr_outliers(dfs["detalleordenes"], "precio_unitario"))

# 🍎 DIRECCIONES ENVIO

add_issue("direccionesenvio", "usuario_id_invalid",
          invalid_categories(dfs["direccionesenvio"], "usuario_id", usuarios_to_analize))

# 🍎 CARRITO

add_issue("carrito", "usuario_id_invalid",
          invalid_categories(dfs["carrito"], "usuario_id", usuarios_to_analize))

add_issue("carrito", "producto_id_invalid",
          invalid_categories(dfs["carrito"], "producto_id", productos_to_analize))

add_issue("carrito", "cantidad_negative",
          negative_values(dfs["carrito"], "cantidad"))

add_issue("carrito", "cantidad_outliers",
          iqr_outliers(dfs["carrito"], "cantidad"))
# 🍎 ORDENES METODOS PAGO

add_issue("ordenesmetodospago", "orden_id_invalid",
          invalid_categories(dfs["ordenesmetodospago"], "orden_id", ordenes_to_analize))

add_issue("ordenesmetodospago", "metodo_pago_id_invalid",
          invalid_categories(dfs["ordenesmetodospago"], "metodo_pago_id", metodos_to_analize))

add_issue("ordenesmetodospago", "monto_pagado_negative",
          negative_values(dfs["ordenesmetodospago"], "monto_pagado"))

add_issue("ordenesmetodospago", "monto_pagado_outliers",
          iqr_outliers(dfs["ordenesmetodospago"], "monto_pagado"))

# 🍎 RESENAS PRODUCTOS

add_issue("resenasproductos", "usuario_id_invalid",
          invalid_categories(dfs["resenasproductos"], "usuario_id", usuarios_to_analize))

add_issue("resenasproductos", "producto_id_invalid",
          invalid_categories(dfs["resenasproductos"], "producto_id", productos_to_analize))

add_issue("resenasproductos", "calificacion_negative",
          negative_values(dfs["resenasproductos"], "calificacion"))

add_issue("resenasproductos", "calificacion_outliers",
          iqr_outliers(dfs["resenasproductos"], "calificacion"))

# 🍎 HISTORIAL PAGOS

add_issue("historialpagos", "orden_id_invalid",
          invalid_categories(dfs["historialpagos"], "orden_id", ordenes_to_analize))

add_issue("historialpagos", "metodo_pago_id_invalid",
          invalid_categories(dfs["historialpagos"], "metodo_pago_id", metodos_to_analize))

add_issue("historialpagos", "monto_negative",
          negative_values(dfs["historialpagos"], "monto"))

add_issue("historialpagos", "monto_outliers",
          iqr_outliers(dfs["historialpagos"], "monto"))

quality_issues


{'productos': {'precio_outliers':     producto_id                      nombre  \
  0             1       Smartphone Galaxy A54   
  1             2     Laptop Dell Inspiron 15   
  2             3  Auriculares Bluetooth Sony   
  9            10    Bicicleta Montaña Aro 29   
  33           34       Consola PlayStation 5   
  
                                            descripcion  precio  stock  \
  0   Teléfono inteligente con pantalla AMOLED y cám...  349.99     50   
  1   Laptop para trabajo y estudio con procesador I...  799.00     30   
  2   Auriculares inalámbricos con cancelación de ru...  129.99    100   
  9          Bicicleta para adulto con cambios Shimano.  299.00     20   
  33       Consola de videojuegos de última generación.  549.99     20   
  
      categoria_id  
  0              1  
  1              1  
  2              1  
  9              4  
  33            12  ,
  'stock_outliers':     producto_id                       nombre  \
  3             4       Camis

✅ Conclusiones generales del análisis de calidad

Después de llevar a cabo todos los procedimientos de diagnóstico, desde la exploración inicial hasta los controles avanzados de integridad, outliers y valores inválidos, se puede concluir que la calidad general de los datos es bastante sólida y no presenta problemas críticos que puedan afectar el análisis o la carga posterior.

El único detalle que se ha encontrado es que varios correos electrónicos tienen un espacio extra, por cómo se generaron en base al nombre. Aunque esto no impacta directamente en las relaciones entre tablas, podría causar inconvenientes en procesos de autenticación o normalización más adelante. Por ahora, se ha decidido mantener los valores tal como vienen del origen, aunque se documentará para una futura etapa de limpieza.

En cuanto a los outliers detectados mediante el método IQR, la mayoría se debe a diferencias naturales entre productos de categorías muy distintas. Por ejemplo, al comparar camisetas de bajo costo con electrodomésticos o dispositivos electrónicos, es normal que el precio o el stock varíen, lo que puede llevar a falsos positivos. En particular, los altos niveles de stock corresponden a productos de alta rotación, como cuadernos o artículos de higiene, mientras que otros productos tienen un stock mucho menor debido a su naturaleza. Por lo tanto, estos “outliers” no son errores, sino variaciones esperadas en el catálogo.

Por último, las validaciones de integridad referencial (FK inválidas), valores negativos y categorías inexistentes no mostraron problemas significativos en ninguna de las tablas. El pipeline puede seguir funcionando con normalidad, y estos hallazgos quedarán registrados para futuras mejoras en el proceso de ingesta y estandarización.